In [51]:
# use xrootd to browse/open files on cern eos
import XRootD.client
import random

In [52]:
path = "root://eoscms.cern.ch/"
xrdurl =  XRootD.client.URL(path)
xrdfs = XRootD.client.FileSystem(xrdurl.hostid)
xrdpath = xrdurl.path
print(xrdpath)

num_clients = 16 # number of xroord clients to load files
xrd_filelist = [f"{xrdfs.url.protocol}://user_{random.randrange(num_clients)}@{xrdfs.url.hostname}:{xrdfs.url.port}/{file}" for file in filelist]

In [53]:
import uproot # see https://uproot.readthedocs.io/en/latest/index.html 
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import pdb
import vector
import hist
from hist import Hist
import os

In [54]:
import argparse
import numpy as np
import awkward as ak
import uproot
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser(prog='./hlt_fit')
parser.add_argument(
    '-i', '--input', nargs='+',
    help='specify input ntuple root files'
)
parser.add_argument(
    '-o', '--output', default='./',
    help='specify output directory'
)
parser.add_argument(
    '--runs', nargs='*', default=None,
    help='specify runs to be processed'
)
args = parser.parse_args()

# acceptance cuts
ptCut = 30
etaCut = 2.5

#mass binning 
mass_low = 76
mass_hi = 106
mass_nBins = 120         

#lumisection binning 
lumi_nBins = 120
lumi_low = 0.5
lumi_hi = 1300.5

#binning for histogram of number of primary ver
pv_nBins = 100
pv_low = 0.5 
pv_hi = 100.5

electronmass = 0.510998946
etaBound = 0.9 

dirOut = args.output
if not os.path.isdir(dirOut):
    print(f"create output directory {dirOut}")
    os.mkdir(dirOut)
    
branches_electron = [
    "Electron_pt","Electron_eta","Electron_phi","Electron_charge","Electron_cutBased"
]

branches_trigger = [
    "TrigObj_id","TrigObj_pt","TrigObj_filterBits","TrigObj_phi","TrigObj_eta"
]
branches_event = ["luminosityBlock","PV_npvs"]


# processing the event in batches to avoid using too much memory 

xrd_filelist = {f : "Events" for f in xrd_filelist}

full_collection = None
runs = [357442]
# Function to calculate ΔR
def delta_r(eta1, phi1, eta2, phi2):
    # Calculate Δφ and account for periodicity
    dphi = np.abs(phi1 - phi2)
    dphi = np.where(dphi > np.pi, 2 * np.pi - dphi, dphi)
    # Calculate ΔR
    return np.sqrt((eta1 - eta2)**2 + dphi**2)

for run in runs:
    print(f"Now at run {run}")
    hists = dict()
    xx_h1 = np.array([])
    yy_h1 = np.array([])
    
    histograms = {
    "h_mass_1hlt_BB": hist.Hist(
        hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"),
        hist.axis.Regular(120, 76, 106, name="mass")
    ,),
    "h_mass_1hlt_BE": hist.Hist(
        hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"),
        hist.axis.Regular(120, 76, 106, name="mass")
    ,),
    "h_mass_1hlt_EE": hist.Hist(
        hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"),
        hist.axis.Regular(120, 76, 106, name="mass")
    ,),
    "h_mass_2hlt_BB": hist.Hist(
        hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"),
        hist.axis.Regular(120, 76, 106, name="mass")
    ,),
    "h_mass_2hlt_BE": hist.Hist(
        hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"),
        hist.axis.Regular(120, 76, 106, name="mass")
    ,),
    "h_mass_2hlt_EE": hist.Hist(
        hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"),
        hist.axis.Regular(120, 76, 106, name="mass")
    ,)
}
    
        
    # Loop through the data in batches
    for batch in uproot.iterate(xrd_filelist,["luminosityBlock", "Electron_pt", "Electron_eta", "Electron_phi", "Electron_charge","Electron_cutBased", "PV_npvs", "TrigObj_id", "TrigObj_pt", "TrigObj_eta", "TrigObj_phi", "TrigObj_filterBits"],filter_name=branches_event + branches_electron + branches_trigger,cut="(run==357442) & HLT_Ele32_WPTight_Gsf & (nElectron>=2)"):
        if len(batch) == 0:
            continue
        print(f"Have a batch with {len(batch)} events")    
        
        events = batch[branches_event]
        
        xx_h1 = np.append(xx_h1,events["luminosityBlock"].to_numpy())
        
        yy_h1 = np.append(yy_h1,events["PV_npvs"].to_numpy())
        
    # electron objects
        electrons = batch[branches_electron]
    
    # rename
    
        electrons["pt"] = electrons["Electron_pt"]
        electrons["eta"] = electrons["Electron_eta"]
        electrons["phi"] = electrons["Electron_phi"]
        electrons["charge"] = electrons["Electron_charge"]
        electrons["cut"] = electrons["Electron_cutBased"]

    # index of the electron within the event (needed to disambiguate)
    
        electrons["index"] = ak.local_index(electrons["Electron_pt"])
    
     # Prepare electron objects
        electrons = ak.zip({
            "pt": batch["Electron_pt"],
            "eta": batch["Electron_eta"],
            "phi": batch["Electron_phi"],
            "charge": batch["Electron_charge"],
            "cut": batch["Electron_cutBased"],
        })

    # Prepare trigger objects
        triggers = ak.zip({
            "pt": batch["TrigObj_pt"],
            "eta": batch["TrigObj_eta"],
            "phi": batch["TrigObj_phi"],
            "id": batch["TrigObj_id"],
            "filterBits": batch["TrigObj_filterBits"],
        })

    # Electrons that pass ID criteria
        id_pass = electrons[(electrons["pt"] > ptCut) &(np.abs(electrons["eta"]) < etaCut) &(electrons["cut"] >= 3) ]

    # Good HLT objects
        goodHLTObj = triggers[(triggers["pt"] > 27) & (triggers["id"] == 11) & ((triggers["filterBits"] & 2) != 0)]

    # Calculate ΔR between id_pass and goodHLTObj
        delta_R_matrix = delta_r(
        electrons["eta"][:, None],  # Broadcasting: Expand id_pass["eta"] along a new axis
        electrons["phi"][:, None],
        goodHLTObj["eta"],
        goodHLTObj["phi"]
    )


    # Example: Find minimum ΔR for each electron
        min_delta_R = ak.min(delta_R_matrix, axis=1)
        
    # collection of electrons that pass ID and hlt
        
        hlt_pass = electrons[(electrons["pt"] > ptCut) & (abs(electrons["eta"]) < etaCut) &(electrons["cut"] >= 3) & (min_delta_R < 0.1)]

    # collection of electrons that pass ID and fail hlt
        
        hlt_fail = electrons[(electrons["pt"] > ptCut) & (abs(electrons["eta"]) < etaCut) &(electrons["cut"] >= 3) & (min_delta_R > 0.1)]
        
        def produce(name, tags, probes=None, disambiguate=False):
            
            BB = []
            BE = []
            EE = []
            
            # 1.) build pairs
            if probes is None:
                p_charge = ak.combinations(tags["charge"], 2)        
                lefts, rights = ak.unzip(p_charge)
                mask_p = lefts*rights == -1

                p_pt = ak.combinations(tags["pt"], 2)[mask_p]
                p_eta = ak.combinations(tags["eta"], 2)[mask_p]
                p_phi = ak.combinations(tags["phi"], 2)[mask_p]
            
            else:              
                p_charge = ak.cartesian([tags["charge"], probes["charge"]])        
                l_charge , r_charge = ak.unzip(p_charge)
                mask_p = l_charge*r_charge == -1
                if disambiguate:
                    p_idx = ak.cartesian([tags["index"], probes["index"]])
                    l_idx, r_idx = ak.unzip(p_idx)
                    mask_p = mask_p & (l_idx != r_idx)

                p_pt = ak.cartesian([tags["pt"],probes["pt"]])[mask_p]
                p_eta = ak.cartesian([tags["eta"],probes["eta"]])[mask_p]
                p_phi = ak.cartesian([tags["phi"],probes["phi"]])[mask_p]
            l_eta, r_eta = ak.unzip(p_eta)

            l_pt , r_pt = ak.unzip(p_pt)

            
            l_phi, r_phi = ak.unzip(p_phi)

            ele1 = vector.obj(pt=l_pt, phi=l_phi, eta=l_eta, mass=l_pt*0+electronmass)
            ele2 = vector.obj(pt=r_pt, phi=r_phi, eta=r_eta, mass=r_pt*0+electronmass)

            masses = (ele1 + ele2).mass
            mask_mass = (masses > mass_low) & (masses < mass_hi)
        
        
        # 3.) Fill histograms    
            
            for region, mask_eta in (
                ("BB", (abs(l_eta) < etaBound) & (abs(r_eta) < etaBound)),
                ("BE", (abs(l_eta) < etaBound) != (abs(r_eta) < etaBound)),
                ("EE", (abs(l_eta) >= etaBound) & (abs(r_eta) >= etaBound))
             ):

                
                yy = masses[mask_eta & mask_mass]   # select tag and probe pairs in eta and mass range
                xx = events["luminosityBlock"]
#                 print(f"\nRegion: {region}")
#                 print(f"Mask count for region '{region}': {len(yy)} events")

                # pair multiplicities in each event 
                # counts = ak.num(yy)                                                                   
                
                # bring the event by event array into the same dimension
                if len(xx) > 0:
                
                    xx = ak.unflatten(xx,counts=1)
                # bring the arrays into the same form
                    xx, yy = ak.broadcast_arrays(xx,yy)
                 
                    xx = ak.flatten(xx).to_numpy()
                
                    yy = ak.flatten(yy).to_numpy()
                    
                    pairs = np.column_stack((xx, yy))
                    
                    if region == "BB":
                        BB.extend(pairs)
                    elif region == "BE":
                        BE.extend(pairs)
                    elif region == "EE":
                        EE.extend(pairs)


            BB, BE, EE = map(lambda x: np.array(x).reshape(-1, 2), (BB, BE, EE))

            
            for region_data , region_name in zip([BB,BE,EE],["BB","BE","EE"]):
                if len(region_data)>0 :
                    xx_region = region_data[:,0] 
                    yy_region = region_data[:,1]
                
                
                    specific_name = f"h_mass_{name}_{region_name}"
    
    
                    histograms[specific_name].fill(xx_region, yy_region)
        

            return histograms
                

        produce("1hlt", hlt_pass,hlt_fail) 
        produce("2hlt",hlt_pass)
        
        # Save histograms
        h1 = hist.Hist(hist.axis.Regular(120, 0.5, 1300.5, name="lumisec"), hist.axis.Regular(100, 0.5, 100.5, name="Npv"),)
        h1.fill(np.array(xx_h1), np.array(yy_h1))
        
        
        file_path = os.path.join(dirOut, "output_resultshlt.root")
        with uproot.recreate(file_path) as root_file:
            root_file["histnpv"] = h1
            for name, hist_obj in histograms.items():
                 root_file[name] = hist_obj
        
        print(f"Results saved to {file_path}")

Now at run 357442
Have a batch with 2001 events
Results saved to ./output_resultshlt.root
Have a batch with 14443 events
Results saved to ./output_resultshlt.root
Have a batch with 1494 events
Results saved to ./output_resultshlt.root
Have a batch with 1200 events
Results saved to ./output_resultshlt.root
Have a batch with 2101 events
Results saved to ./output_resultshlt.root
Have a batch with 1982 events
Results saved to ./output_resultshlt.root
Have a batch with 5639 events
Results saved to ./output_resultshlt.root
Have a batch with 5355 events
Results saved to ./output_resultshlt.root
Have a batch with 3974 events
Results saved to ./output_resultshlt.root
Have a batch with 2361 events
Results saved to ./output_resultshlt.root
Have a batch with 2987 events
Results saved to ./output_resultshlt.root
Have a batch with 8710 events
Results saved to ./output_resultshlt.root
Have a batch with 2641 events
Results saved to ./output_resultshlt.root
Have a batch with 1788 events
Results saved t

OSError: XRootD error: [ERROR] Operation expired
in file root://user_7@eoscms.cern.ch:1094//store/data/Run2022C/EGamma/NANOAOD/22Sep2023-v1/40000/53d8b7cc-9394-4e14-b9a7-5fa9dc475ebd.root

In [ ]:
import ROOT
import numpy as np
import uproot
import hist
import os
import awkward as ak  # To handle arrays in `TTree` format

# Define the list of histogram names you want to process
hist_names = ["h_mass_2hlt_EE", "h_mass_1hlt_EE"]

# Open the ROOT file
demo_file = "output_resultshlt.root"

# Define an output dictionary to store the processed results for each histogram
processed_results = {}

with uproot.open(demo_file) as root_file_2:
    # Iterate over each histogram name
    for hist_name in hist_names:
        # Access the histogram data for the current component
        uproot_hist = root_file_2[hist_name]
        values, edges_lumisec, edges_mass = uproot_hist.to_numpy()

        # Define bin ranges for slicing
        x_bin_start1 = 1    # This is the first bin in your selection (second bin in ROOT convention)
        x_bin_end1 = 41

        x_bin_start2 = 42   # This is the first bin in your second selection (second bin in ROOT convention)
        x_bin_end2 = 83
        
        x_bin_start3 = 83   # This is the first bin in your third selection (second bin in ROOT convention)
        x_bin_end3 = 123

        # Slice and project the data based on the bin ranges
        sliced_values1 = values[x_bin_start1:x_bin_end1 + 1, :]
        sliced_values2 = values[x_bin_start2:x_bin_end2 + 1, :]
        sliced_values3 = values[x_bin_start3:x_bin_end3 + 1, :]

        projected_values1 = sliced_values1.sum(axis=0)
        projected_values2 = sliced_values2.sum(axis=0)
        projected_values3 = sliced_values3.sum(axis=0)

        # Define y-axis bin edges, which remain the same
        y_bin_edges = edges_mass

        # Create a fresh histogram for each `hist_name` to avoid reusing
        myHist1 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist1.view()[()] = projected_values1
        
        myHist2 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist2.view()[()] = projected_values2
        
        myHist3 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist3.view()[()] = projected_values3

        # Store processed projections and histograms in the results dictionary
        processed_results[hist_name] = {
            "projected_values1": projected_values1,
            "projected_values2": projected_values2,
            "projected_values3": projected_values3,
            "histogram1": myHist1,
            "histogram2": myHist2,
            "histogram3": myHist3,
        }

        # Optionally print for each histogram
        print(f"Processed {hist_name}")
        print("Projected values 1:", projected_values1)
        print("Projected values 2:", projected_values2)
        print("Projected values 3:", projected_values3)

# Example of saving to a new ROOT file
dirOut = "mass_regionEE_hlt.root"  # Specify your output directory
# os.makedirs(dirOut, exist_ok=True)
file_path = os.path.join(dirOut)
# Write processed histograms and projections to the ROOT file
with uproot.recreate(file_path) as root_file:
    for hist_name, result in processed_results.items():  
        # Save each histogram separately
        root_file[f"{hist_name}_hist1"] = result["histogram1"]
        root_file[f"{hist_name}_hist2"] = result["histogram2"]
        root_file[f"{hist_name}_hist3"] = result["histogram3"]

print("All histograms processed and saved.")




In [ ]:
import ROOT
import numpy as np
import uproot
import hist
import os
import awkward as ak  # To handle arrays in `TTree` format

# Define the list of histogram names you want to process
hist_names = ["h_mass_2hlt_BB", "h_mass_1hlt_BB"]

# Open the ROOT file
demo_file = "output_resultshlt.root"

# Define an output dictionary to store the processed results for each histogram
processed_results = {}

with uproot.open(demo_file) as root_file_2:
    # Iterate over each histogram name
    for hist_name in hist_names:
        # Access the histogram data for the current component
        uproot_hist = root_file_2[hist_name]
        values, edges_lumisec, edges_mass = uproot_hist.to_numpy()

        # Define bin ranges for slicing
        x_bin_start1 = 1    # This is the first bin in your selection (second bin in ROOT convention)
        x_bin_end1 = 41

        x_bin_start2 = 42   # This is the first bin in your second selection (second bin in ROOT convention)
        x_bin_end2 = 83
        
        x_bin_start3 = 83   # This is the first bin in your third selection (second bin in ROOT convention)
        x_bin_end3 = 123

        # Slice and project the data based on the bin ranges
        sliced_values1 = values[x_bin_start1:x_bin_end1 + 1, :]
        sliced_values2 = values[x_bin_start2:x_bin_end2 + 1, :]
        sliced_values3 = values[x_bin_start3:x_bin_end3 + 1, :]

        projected_values1 = sliced_values1.sum(axis=0)
        projected_values2 = sliced_values2.sum(axis=0)
        projected_values3 = sliced_values3.sum(axis=0)

        # Define y-axis bin edges, which remain the same
        y_bin_edges = edges_mass

        # Create a fresh histogram for each `hist_name` to avoid reusing
        myHist1 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist1.view()[()] = projected_values1
        
        myHist2 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist2.view()[()] = projected_values2
        
        myHist3 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist3.view()[()] = projected_values3

        # Store processed projections and histograms in the results dictionary
        processed_results[hist_name] = {
            "projected_values1": projected_values1,
            "projected_values2": projected_values2,
            "projected_values3": projected_values3,
            "histogram1": myHist1,
            "histogram2": myHist2,
            "histogram3": myHist3,
        }

        # Optionally print for each histogram
        print(f"Processed {hist_name}")
        print("Projected values 1:", projected_values1)
        print("Projected values 2:", projected_values2)
        print("Projected values 3:", projected_values3)

# Example of saving to a new ROOT file
dirOut = "mass_regionBB_hlt.root"  # Specify your output directory
# os.makedirs(dirOut, exist_ok=True)
file_path = os.path.join(dirOut)
# Write processed histograms and projections to the ROOT file
with uproot.recreate(file_path) as root_file:
    for hist_name, result in processed_results.items():  
        # Save each histogram separately
        root_file[f"{hist_name}_hist1"] = result["histogram1"]
        root_file[f"{hist_name}_hist2"] = result["histogram2"]
        root_file[f"{hist_name}_hist3"] = result["histogram3"]

print("All histograms processed and saved.")




In [ ]:
import ROOT
import numpy as np
import uproot
import hist
import os
import awkward as ak  # To handle arrays in `TTree` format

# Define the list of histogram names you want to process
hist_names = ["h_mass_2hlt_BE", "h_mass_1hlt_BE"]

# Open the ROOT file
demo_file = "output_resultshlt.root"

# Define an output dictionary to store the processed results for each histogram
processed_results = {}

with uproot.open(demo_file) as root_file_2:
    # Iterate over each histogram name
    for hist_name in hist_names:
        # Access the histogram data for the current component
        uproot_hist = root_file_2[hist_name]
        values, edges_lumisec, edges_mass = uproot_hist.to_numpy()

        # Define bin ranges for slicing
        x_bin_start1 = 1    # This is the first bin in your selection (second bin in ROOT convention)
        x_bin_end1 = 41

        x_bin_start2 = 42   # This is the first bin in your second selection (second bin in ROOT convention)
        x_bin_end2 = 83
        
        x_bin_start3 = 83   # This is the first bin in your third selection (second bin in ROOT convention)
        x_bin_end3 = 123

        # Slice and project the data based on the bin ranges
        sliced_values1 = values[x_bin_start1:x_bin_end1 + 1, :]
        sliced_values2 = values[x_bin_start2:x_bin_end2 + 1, :]
        sliced_values3 = values[x_bin_start3:x_bin_end3 + 1, :]

        projected_values1 = sliced_values1.sum(axis=0)
        projected_values2 = sliced_values2.sum(axis=0)
        projected_values3 = sliced_values3.sum(axis=0)

        # Define y-axis bin edges, which remain the same
        y_bin_edges = edges_mass

        # Create a fresh histogram for each `hist_name` to avoid reusing
        myHist1 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist1.view()[()] = projected_values1
        
        myHist2 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist2.view()[()] = projected_values2
        
        myHist3 = hist.Hist(hist.axis.Regular(120, 76, 106, name="mass"))
        myHist3.view()[()] = projected_values3

        # Store processed projections and histograms in the results dictionary
        processed_results[hist_name] = {
            "projected_values1": projected_values1,
            "projected_values2": projected_values2,
            "projected_values3": projected_values3,
            "histogram1": myHist1,
            "histogram2": myHist2,
            "histogram3": myHist3,
        }

        # Optionally print for each histogram
        print(f"Processed {hist_name}")
        print("Projected values 1:", projected_values1)
        print("Projected values 2:", projected_values2)
        print("Projected values 3:", projected_values3)

# Example of saving to a new ROOT file
dirOut = "mass_regionBE_hlt.root"  # Specify your output directory
# os.makedirs(dirOut, exist_ok=True)
file_path = os.path.join(dirOut)
# Write processed histograms and projections to the ROOT file
with uproot.recreate(file_path) as root_file:
    for hist_name, result in processed_results.items():  
        # Save each histogram separately
        root_file[f"{hist_name}_hist1"] = result["histogram1"]
        root_file[f"{hist_name}_hist2"] = result["histogram2"]
        root_file[f"{hist_name}_hist3"] = result["histogram3"]

print("All histograms processed and saved.")


